# Анализ паттернов преобразования предложений

Пригодится когда я буду делать поиск закономерностей в большом корпусе текстов

In [ ]:
import spacy
from typing import Dict, List, Tuple, Set
from collections import defaultdict

nlp = spacy.load("en_core_web_sm")

def analyze_sentence_pattern(doc):
    """Анализирует структуру предложения и извлекает ключевые паттерны"""
    patterns = {
        'nouns': [],
        'verbs': [],
        'prepositions': [],
        'dependencies': [],
        'noun_phrases': [],
        'verb_phrases': []
    }
    
    for token in doc:
        # Основные части речи
        if token.pos_ == "NOUN":
            patterns['nouns'].append({
                'text': token.text,
                'lemma': token.lemma_,
                'dep': token.dep_,
                'head': token.head.text,
                'children': [child.text for child in token.children]
            })
        elif token.pos_ == "VERB":
            patterns['verbs'].append({
                'text': token.text,
                'lemma': token.lemma_,
                'dep': token.dep_,
                'tense': token.morph.get('Tense', []),
                'voice': token.morph.get('Voice', [])
            })
        elif token.pos_ == "ADP":
            patterns['prepositions'].append({
                'text': token.text,
                'dep': token.dep_,
                'object': token.head.text
            })
        
        # Зависимости
        patterns['dependencies'].append({
            'token': token.text,
            'dep': token.dep_,
            'head': token.head.text,
            'pos': token.pos_
        })
    
    return patterns

def extract_noun_phrases(doc):
    """Извлекает именные группы"""
    noun_phrases = []
    for chunk in doc.noun_chunks:
        noun_phrases.append({
            'text': chunk.text,
            'root': chunk.root.text,
            'deps': [(t.text, t.dep_) for t in chunk]
        })
    return noun_phrases

def find_transformation_pattern(source_doc, target_doc):
    """Находит паттерн преобразования между исходным и целевым предложениями"""
    
    source_patterns = analyze_sentence_pattern(source_doc)
    target_patterns = analyze_sentence_pattern(target_doc)
    
    # Анализ ключевых изменений
    transformations = {
        'noun_changes': [],
        'verb_changes': [],
        'structure_changes': [],
        'voice_changes': []
    }
    
    # 1. Анализ существительных
    source_nouns = {n['text'].lower(): n for n in source_patterns['nouns']}
    target_nouns = {n['text'].lower(): n for n in target_patterns['nouns']}
    
    for noun in target_nouns:
        if noun in source_nouns:
            # Существительное сохранилось, но изменилась роль
            transformations['noun_changes'].append({
                'noun': noun,
                'source_role': source_nouns[noun]['dep'],
                'target_role': target_nouns[noun]['dep']
            })
    
    # 2. Анализ глаголов
    source_verbs = [v['lemma'] for v in source_patterns['verbs']]
    target_verbs = [v['lemma'] for v in target_patterns['verbs']]
    
    # Поиск связи между существительными и глаголами
    for target_verb in target_verbs:
        for source_noun in source_patterns['nouns']:
            if target_verb in source_noun['lemma'] or source_noun['lemma'] in target_verb:
                transformations['verb_changes'].append({
                    'source_noun': source_noun['text'],
                    'target_verb': target_verb,
                    'transformation': f"NOUN({source_noun['text']}) -> VERB({target_verb})"
                })
    
    # 3. Анализ структуры предложения
    source_structure = [d['dep'] for d in source_patterns['dependencies']]
    target_structure = [d['dep'] for d in target_patterns['dependencies']]
    
    transformations['structure_changes'] = {
        'source_structure': source_structure,
        'target_structure': target_structure
    }
    
    return transformations

def print_pattern_analysis(source_text, target_text):
    """Выводит подробный анализ паттернов"""
    source_doc = nlp(source_text)
    target_doc = nlp(target_text)
    
    print("=== АНАЛИЗ ПАТТЕРНОВ ПРЕОБРАЗОВАНИЯ ===")
    print(f"Исходное: {source_text}")
    print(f"Целевое: {target_text}")
    print()
    
    # Структура исходного предложения
    print("ИСХОДНОЕ ПРЕДЛОЖЕНИЕ:")
    for token in source_doc:
        print(f"  {token.text:<15} {token.pos_:<8} {token.dep_:<10} -> {token.head.text}")
    print()
    
    # Структура целевого предложения
    print("ЦЕЛЕВОЕ ПРЕДЛОЖЕНИЕ:")
    for token in target_doc:
        print(f"  {token.text:<15} {token.pos_:<8} {token.dep_:<10} -> {token.head.text}")
    print()
    
    # Паттерны преобразования
    patterns = find_transformation_pattern(source_doc, target_doc)
    
    print("ПАТТЕРНЫ ПРЕОБРАЗОВАНИЯ:")
    for change_type, changes in patterns.items():
        if changes:
            print(f"  {change_type}:")
            for change in changes:
                print(f"    {change}")
    print()
    
    # Специфический анализ для твоего примера
    print("СПЕЦИФИЧЕСКИЙ АНАЛИЗ:")
    
    # Поиск связи "refutation" -> "refuted"
    source_refutation = None
    for token in source_doc:
        if 'refut' in token.lemma_.lower():
            source_refutation = token
            break
    
    target_refuted = None
    for token in target_doc:
        if 'refut' in token.lemma_.lower():
            target_refuted = token
            break
    
    if source_refutation and target_refuted:
        print(f"  NOUN -> VERB: '{source_refutation.text}' -> '{target_refuted.text}'")
        print(f"  Лемма: {source_refutation.lemma_} -> {target_refuted.lemma_}")
        print(f"  Морфология: {source_refutation.morph} -> {target_refuted.morph}")
    
    # Анализ "aging mechanisms"
    source_aging = None
    for token in source_doc:
        if token.text.lower() == 'aging':
            source_aging = token
            break
    
    if source_aging:
        print(f"  'aging mechanisms' сохраняется как подлежащее")
    
    # Анализ залога
    target_voice = None
    for token in target_doc:
        if token.pos_ == "VERB":
            voice = token.morph.get('Voice', [])
            if voice:
                target_voice = voice[0]
                break
    
    if target_voice:
        print(f"  Залог: {target_voice}")

# Тестирование

source_text = "Methods of Refutation of Aging Mechanisms for the Purposes of Biological Immortality"
target_text = "aging mechanisms have been refuted"

print_pattern_analysis(source_text, target_text)